<img src="fifa.jpeg" style="float:left">

In [ ]:
!pip install refractml

In [ ]:
!pip freeze | grep numpy

In [ ]:
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

!pip uninstall cloudpickle -y
!pip install cloudpickle==2.2.1

# Fifa 19 Dataset - Transfer Value predicting Regression Model

# Importing required Packages

In [1]:

import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from fosforml import *
from fosforml.constants import MLModelFlavours

import warnings
warnings.filterwarnings('ignore')

<ipython-input-1-15b4a69e8c10>:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Loading Dataset

In [2]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

,short_name,nationality,overall,potential,wage_eur,value_eur,age,height_cm,weight_kg,attacking_crossing,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,Argentina,94,94,565000,95500000,32,170,72,88,...,75,96,33,37,26,6,11,15,14,8
1,Cristiano Ronaldo,Portugal,93,93,405000,58500000,34,187,83,84,...,85,95,28,32,24,7,11,15,14,11
2,Neymar Jr,Brazil,92,92,290000,105500000,27,175,68,87,...,90,94,27,26,29,9,9,15,15,11
3,J. Oblak,Slovenia,91,93,125000,77500000,26,188,87,13,...,11,68,27,12,18,87,92,78,90,89
4,E. Hazard,Belgium,91,91,470000,90000000,28,175,74,81,...,88,91,34,27,22,11,12,6,8,8
5,K. De Bruyne,Belgium,91,91,370000,90000000,28,181,70,93,...,79,91,68,58,51,15,13,5,10,13
6,M. ter Stegen,Germany,90,93,250000,67500000,27,187,85,18,...,25,70,25,13,10,88,85,88,88,90
7,V. van Dijk,Netherlands,90,91,200000,78000000,27,193,92,53,...,62,89,91,92,85,13,10,13,11,11
8,L. Modric,Croatia,90,90,340000,45000000,33,172,66,86,...,82,92,68,76,71,13,9,7,14,9
9,M. Salah,Egypt,90,90,240000,80500000,27,175,71,79,...,77,91,38,43,41,14,14,9,11,14


# Seperating Dataset into Training and Test

In [3]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

# Training the Model

In [4]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

GradientBoostingRegressor()

# Defining a Score Function

In [5]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()


# Testing Score Function

In [6]:
# to verify if score func works

import requests
req = requests.Request()
req.json = {"payload":X_test}

'''
Reference Sample Payload 

req.json = {"payload": [ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9,
        10,  79,  82]}
        
'''
score(gbm_default, req)

[7371874.6616814695,
 4738569.183719677,
 21420646.724212058,
 18565368.234815322,
 2521068.456590732,
 4927893.686116417,
 17717792.640952602,
 536649.2767679255,
 5379405.813943993,
 2782661.6857576417,
 3109098.804303742,
 3854078.9689723514,
 4098721.55638351,
 7769807.652539339,
 4692304.372048858,
 16930281.548238754,
 3473019.6617231485,
 6661265.394274316,
 3016063.327552542,
 10421430.298660973,
 1978920.7122082596,
 2802425.7801835784,
 905938.3801071717,
 7703026.231638118,
 4737163.454637376,
 9200545.134584036,
 40671774.995375425,
 11732600.005104976,
 11769365.628129423,
 127060.88757149575,
 5192798.597258351,
 3933793.16651854,
 2461772.794033602,
 17703016.395034228,
 5660456.160853367,
 2502423.505083093,
 6259869.183645455,
 8698457.57909251,
 1833849.8102294612,
 4711891.257815066,
 2820859.9746349435,
 2385315.8941515298,
 1986428.045896931,
 7220836.17030125,
 2028027.9431738039,
 1953495.1905439016,
 3606521.7807429815,
 2197176.334081368,
 6941085.884180219,
 5

# Calculating Y_true and Y_pred

In [ ]:
Y_true = y_test

Y_pred = gbm_default.predict(X_test)
features = list(X_train)

# Registering the Model

In [ ]:
register_model(
    gbm_default,
    score,
    "fifa_model_310_fosforml",
    "fifa_regression",
    MLModelFlavours.sklearn,
    init_script="mkdir abc \\n cd abc \\n touch abc.txt",
    input_type="json",
    explain_ai=True,
    x_train=X_train, 
    y_train=y_train, 
    y_true=Y_true,
    y_pred=Y_pred,
    feature_names=features,
    feature_ids=features,
    target_names=["Value"],
    model_type="regression"
)

In [ ]:
# def pickle_dumps(obj, file_path):
#     import cloudpickle
#     out = open(file_path, "wb")
#     cloudpickle.dump(obj, out)
#     out.close()

In [ ]:
# pickle_dumps(gbm_default, "/notebooks/notebooks/ml_model_2")

In [ ]:
X_train

In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(gbm_default, "/notebooks/notebooks/ml_model_3")